# Trimming columns a bit more and resampling data

In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.tree import DecisionTreeClassifier,plot_tree
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, accuracy_score,precision_score,f1_score, confusion_matrix,roc_auc_score
from sklearn.multiclass import BaseEstimator
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier,AdaBoostClassifier,VotingClassifier
pd.set_option('display.max_columns', None)
import os 
import sys
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
path="../../Data/"
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
from src.data_cleaning import processing_functions as pfs
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC

### Import prefiltered data 

In [2]:
X_train, X_test, y_train, y_test, classes_dict = pfs.processed_dataset()

### Time to clean up some colunms

In [3]:
X_train=X_train.drop(["amount_tsh","extraction_type_group","management","construction_year","population","gps_height"],axis=1)
X_test=X_test.drop(["amount_tsh","extraction_type_group","management","construction_year","population","gps_height"],axis=1)

KeyError: "['amount_tsh' 'extraction_type_group' 'gps_height'] not found in axis"

### Did some more research into these columns and deemed them not needed due to far to many zero values or thinking they did not provide any actual insight into the pumps functionality.

### amount_tsh=distance from water in basin/source to top. It may play a role but to many zero values with no clear definition of this value

### extraction_type_group=disregarding due to duplicate column

### management= I may bin these based there groups when brining back the funders and installers.

### construction year=many zero values in this columns may need to average/meadians this values to fill the unknowns

### population=dropped this because many zero values. Also from places with no people have pipes that are functionaly so who is reporting this site just very questionable not sure yet.

### gps_height=this may tie into amount_tsh  but for now need to background on it

In [4]:
# def identifying_feature_types(df):
#     """This function lists out the names of columns
#     in a dataframe whose dtypes are objects as categorical
#     features and the numeric types as numeric features
    
#     Returns two lists of columns names first for numeric 
#     and the second for categorical features
#     """
    
#     numeric_features = []
#     categorical_features = []
#     for name, dtype in zip(df.dtypes.index, df.dtypes):
#         if str(dtype).startswith('ob'):
#             categorical_features.append(name)
#         else:
#             numeric_features.append(name)
    
#     return numeric_features, categorical_features

In [5]:
# def ohe_all_categorical_features(df):
#     """This function takes in a dataframe, identifies the
#     dtypes in the dataframe and uses the object dtypes to
#     list out categorical columns
    
#     Next it use OneHotEncoder to convert those Categorical 
#     features
    
#     Returns: the transformed dataframe and a dictionary 
#     containing the ohe object that can be used later to 
#     transform the testing dataset
#     """
    
    
#     #helper function to identify categorical feature names
#     num_feats, cat_feats = identifying_feature_types(df)
    
#     #reassuring the values in categorical features are str types
#     for name in cat_feats:
#         df[name] = df[name].astype(str)
    
#     #use helper function in loop to transform dataframe
#     encoders = {}
    
#     for name in cat_feats:
#         df, ohe = one_hot_encode_feature(df, name)
#         encoders[name] = ohe
    
#     return df, encoders


### Load target data

In [6]:
target_df=pd.read_csv(path+'training_set_labels.csv')
target_df

,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional
...,...,...
59395,60739,functional
59396,27263,functional
59397,37057,functional
59398,31282,functional


### looking at X_train

In [7]:
X_train

,longitude,latitude,basin,region,district_code,lga,population,public_meeting,scheme_management,permit,construction_year,extraction_type,management,payment,water_quality,quantity,source,waterpoint_type,gps_height_binned
29193,33.037555,-2.411289,Lake Victoria,Mwanza,8,Ilemela,877.777778,True,VWC,True,2000,nira/tanira,vwc,pay monthly,soft,insufficient,shallow well,hand pump,above surface up to 1250
35859,30.254171,-4.530045,Lake Tanganyika,Kigoma,2,Kasulu,650.000000,True,Water authority,True,2002,gravity,water board,pay annually,soft,enough,river,communal standpipe,above surface up to 1250
57006,31.515446,-3.335620,Lake Tanganyika,Shinyanga,5,Bukombe,281.600417,True,WUG,True,2000,nira/tanira,wug,unknown,soft,enough,shallow well,hand pump,above surface up to 1250
15959,36.838153,-3.650579,Pangani,Manyara,4,Simanjiro,200.000000,True,unknown,True,2003,other,wug,unknown,salty,insufficient,shallow well,other,above surface up to 1250
21582,33.540494,-9.162976,Lake Nyasa,Mbeya,4,Rungwe,266.138968,True,VWC,unknown,2000,gravity,vwc,never pay,soft,insufficient,spring,communal standpipe,above surface up to 1250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18523,31.367762,-7.786972,Lake Rukwa,Rukwa,3,Nkasi,150.000000,unknown,VWC,False,1984,gravity,vwc,never pay,unknown,enough,river,communal standpipe,above 1750
44739,31.827198,-8.391348,Lake Rukwa,Rukwa,2,Sumbawanga Rural,100.000000,True,VWC,False,1991,other,vwc,never pay,soft,dry,shallow well,other,above surface upto 1750
47734,32.940920,-9.166375,Lake Rukwa,Mbeya,6,Mbozi,387.429369,True,VWC,True,2000,gravity,vwc,never pay,soft,enough,spring,other,above surface up to 1250
40259,35.009905,-10.938834,Lake Nyasa,Ruvuma,3,Mbinga,60.000000,unknown,Water Board,True,2002,gravity,water board,pay monthly,soft,enough,spring,communal standpipe,above surface upto 1750


### Time to encode train data

In [8]:
X_train, encoders=pfs.ohe_all_categorical_features(X_train)

In [9]:
X_train

,longitude,latitude,population,construction_year,Internal,Lake Nyasa,Lake Rukwa,Lake Tanganyika,Lake Victoria,Pangani,Rufiji,Ruvuma / Southern Coast,Wami / Ruvu,Arusha,Dar es Salaam,Dodoma,Iringa,Kagera,Kigoma,Kilimanjaro,Lindi,Manyara,Mara,Mbeya,Morogoro,Mtwara,Mwanza,Pwani,Rukwa,Ruvuma,Shinyanga,Singida,Tabora,Tanga,Arusha Rural,Arusha Urban,Babati,Bagamoyo,Bahi,Bariadi,Biharamulo,Bukoba Rural,Bukoba Urban,Bukombe,Bunda,Chamwino,Chato,Chunya,Dodoma Urban,Geita,Hai,Hanang,Handeni,Igunga,Ilala,Ileje,Ilemela,Iramba,Iringa Rural,Kahama,Karagwe,Karatu,Kasulu,Kibaha,Kibondo,Kigoma Rural,Kigoma Urban,Kilindi,Kilolo,Kilombero,Kilosa,Kilwa,Kinondoni,Kisarawe,Kishapu,Kiteto,Kondoa,Kongwa,Korogwe,Kwimba,Kyela,Lindi Rural,Lindi Urban,Liwale,Longido,Ludewa,Lushoto,Mafia,Magu,Makete,Manyoni,Masasi,Maswa,Mbarali,Mbeya Rural,Mbinga,Mbozi,Mbulu,Meatu,Meru,Misenyi,Missungwi,Mkinga,Mkuranga,Monduli,Morogoro Rural,Morogoro Urban,Moshi Rural,Moshi Urban,Mpanda,Mpwapwa,Mtwara Rural,Mtwara Urban,Mufindi,Muheza,Muleba,Musoma Rural,Mvomero,Mwanga,Nachingwea,Namtumbo,Nanyumbu,Newala,Ngara,Ngorongoro,Njombe,Nkasi,Nyamagana,Nzega,Pangani,Rombo,Rorya,Ruangwa,Rufiji,Rungwe,Same,Sengerema,Serengeti,Shinyanga Rural,Shinyanga Urban,Siha,Sikonge,Simanjiro,Singida Rural,Singida Urban,Songea Rural,Songea Urban,Sumbawanga Rural,Sumbawanga Urban,Tabora Urban,Tandahimba,Tanga,Tarime,Temeke,Tunduru,Ukerewe,Ulanga,Urambo,Uyui,False,True,unknown,Company,Other,Parastatal,Private operator,SWC,Trust,VWC,WUA,WUG,Water Board,Water authority,unknown,False,True,unknown,afridev,cemo,climax,gravity,india mark ii,india mark iii,ksb,mono,nira/tanira,other,other - mkulima/shinyanga,other - play pump,other - rope pump,other - swn 81,submersible,swn 80,walimi,windmill,company,other,other - school,parastatal,private operator,trust,unknown,vwc,water authority,water board,wua,wug,never pay,other,pay annually,pay monthly,pay per bucket,pay when scheme fails,unknown,coloured,fluoride,fluoride abandoned,milky,salty,salty abandoned,soft,unknown,dry,enough,insufficient,seasonal,unknown,dam,hand dtw,lake,machine dbh,other,rainwater harvesting,river,shallow well,spring,unknown,cattle trough,communal standpipe,communal standpipe multiple,dam,hand pump,improved spring,other,above 1750,above surface up to 1250,above surface upto 1750,above surface upto 750,at surface level,below surface level,0,1,13,2,23,3,30,33,4,43,5,53,6,60,62,63,67,7,8,80
29193,33.037555,-2.411289,877.777778,2000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
35859,30.254171,-4.530045,650.000000,2002,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

### checking out our y train values

In [10]:
y_train.values

array([0, 0, 0, ..., 0, 0, 0])

### Lets look at our target values now

In [11]:
unique, counts=np.unique(y_train,return_counts=True)
y_trainvals={ k:v for (k,v) in zip(unique,counts)}
y_trainvals

{0: 24249, 1: 3165, 2: 17136}

### Now we will attemp to fix this imbalance

In [18]:
sm=SMOTE(sampling_strategy='minority',random_state=2020)
X_train_res, y_train_res =sm.fit_resample(X_train.values,y_train,)

In [19]:

unique, counts_res=np.unique(y_train_res,return_counts=True)
y_trainvals1={ k:v for (k,v) in zip(unique,counts_res)}
y_trainvals1

{0: 24249, 1: 24249, 2: 17136}

In [20]:
resample_y_data=y_train_res.sample(n=44550)
resample_y_data
unique, counts_res2=np.unique(resample_y_data,return_counts=True)
y_trainvals2={ k:v for (k,v) in zip(unique,counts_res2)}
y_trainvals2

{0: 16510, 1: 16459, 2: 11581}

In [21]:
print("Basis target values",y_trainvals)
print("Balanced target values",y_trainvals1)

Basis target values {0: 24249, 1: 3165, 2: 17136}
Balanced target values {0: 24249, 1: 24249, 2: 17136}


### The above transformed data added 20k rows so we need to resize the data

In [22]:
resample_y_data=y_train_res.sample(n=44550)
resample_y_data

56664    1
7889     0
30278    2
34428    0
27183    0
        ..
20052    2
26142    0
36144    0
48075    1
15069    0
Name: target, Length: 44550, dtype: int32

In [26]:
X=X_train
y=resample_y_data

### Refittin data just to play around with the data

In [27]:
X_train2, X_test, y_train, y_test =train_test_split(X,y,random_state=2020)

In [28]:
y_train

42714    0
63451    1
48319    1
53877    1
53075    1
        ..
6163     2
23274    0
15819    2
51878    1
27402    0
Name: target, Length: 33412, dtype: int32

### trying to do a decision tree model since we had luck with that last time

In [29]:
dt=DecisionTreeClassifier()
dt.fit(X_train2, y_train)

DecisionTreeClassifier()

In [30]:
dt.score(X_test,y_test)

0.34611240797270604

In [31]:
dt.score(X_train2,y_train)

0.991829282892374

### trying a random tree classifer

In [32]:
#ensble of decision trees
rf=RandomForestClassifier(n_estimators=10)
rf.fit(X_train2,y_train)

RandomForestClassifier(n_estimators=10)

In [33]:
rf.score(X_test,y_test)

0.3452145807146705

### trying abaggin classifer

In [34]:
#bagging
# each bag contains 50% of traning data with 20 decision trees
bg=BaggingClassifier(DecisionTreeClassifier(), max_samples=.5, max_features=1.0,n_estimators=20)
bg.fit(X_train2,y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(), max_samples=0.5,
                  n_estimators=20)

In [35]:
bg.score(X_test,y_test)

0.3462021906985096

### boosting example using data

In [36]:
#Generally over fits and vairance of a model
adb=AdaBoostClassifier(DecisionTreeClassifier(), n_estimators=10,learning_rate=1)
adb.fit(X_train2,y_train)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), learning_rate=1,
                   n_estimators=10)

In [37]:
adb.score(X_test,y_test)

0.3409947926019034

In [38]:
adb.score(X_train2,y_train)

0.991829282892374

### Starting the process of voting class classifer

In [39]:
lr=LogisticRegression()
dt=DecisionTreeClassifier()
svm=SVC(kernel="poly",degree=2)

In [40]:
#voting =hard mean votes on labels no probability
evc=VotingClassifier(estimators=[("lr",lr),("dt",dt),("svm",svm)],voting="hard")

In [41]:
evc.fit(X_train2.iloc[1:4000],y_train.iloc[1:4000])

C:\Users\jstep\anaconda3\envs\Tanzania\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('dt', DecisionTreeClassifier()),
                             ('svm', SVC(degree=2, kernel='poly'))])

In [42]:
evc.score(X_test,y_test)

0.3672113485365416

In [43]:
evc.estimators

[('lr', LogisticRegression()),
 ('dt', DecisionTreeClassifier()),
 ('svm', SVC(degree=2, kernel='poly'))]

Need to work on cleaning and filteing data better have a long way to go.